<a href="https://colab.research.google.com/github/whorseman/Assignments/blob/main/homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following assignment consists again of a theoretical part (learning portfolio) and a practical part (assignment). The goal is to train a neural model from scratch, as we did a few weeks ago. I would like us to repeat the most important things so that we can consolidate our knowledge in this area, for this you will train a model on a dataset that I will provide via OLAT.

The plan would be that in the first we will discuss your learnings from the theory part, that means you are relatively free to fill your Learning Portfolio on this topic and in the following week we will discuss your solutions of the Classification Model.

#Theory part (filling your Learning Portfolio, May 24)

In preparation for the practical part, I ask you to familiarize yourself with the following video sources in the next week:

1) Please watch the following video until random forests:

https://course.fast.ai/Lessons/lesson5.html

2) Please download the following notebooks and edit it in Google-Colab. Take notes and update your Learning Portfolio.

https://www.kaggle.com/code/jhoward/linear-model-and-neural-net-from-scratch

https://www.kaggle.com/code/jhoward/why-you-should-use-a-framework

The contents should be mostly known to you, because these contents are also based on chapter 4, which we have already worked on. We repeat the basics again with a new data set to consolidate our knowledge.



#Practical part (Assignment, May 31)

The following task is a binary classification task. The first column is our target. All variables are categorical variables from which you have to create dummy variables. The target column has only the property Q or F, which is to be modeled. As always, please create a train and a test data set, e.g. 80:20 or 90:10. Your goal is to create a neural network that best predicts the target column. Use probabilities with the sigmoid function as discussed in the theory. When designing your neural network, play with a few different numbers and sizes of layers and different activation functions.

To validate your model, you can compare your results with a logistic regression model. (Note: This data set should allow you to achieve very high accuracies: > 98% maybe even >99%) You can see how these results vary when you use 50% for testing and 50% for training.

In [ ]:
import pandas as pd
from google.colab import drive
from google.colab import data_table
import matplotlib.pyplot as plt
data_table.enable_dataframe_formatter()

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


When we have discussed the assignment, I will tell you what the data set is about, which has its origin from our nature.

In [ ]:
data  = pd.read_csv("/content/drive/MyDrive/DIGO/data_homework_6.csv")
data = data.drop(columns=['variable_16'])
doh   = pd.get_dummies(data)



In [ ]:

cols = list(data)

df2 = np.unique(data[cols].values)

for col in data:
    print(data[col].unique())

['Q' 'F']
['Y' 'C' 'T' 'G' 'L' 'D']
['T' 'Z' 'G' 'H']
['O' 'Z' 'X' 'H' 'F' 'Q' 'C' 'V' 'D' 'S']
['U' 'G']
['Q' 'B' 'M' 'O' 'G' 'D' 'Z' 'T' 'N']
['G' 'B']
['D' 'X']
['O' 'C']
['L' 'O' 'H' 'Q' 'X' 'I' 'V' 'F' 'C' 'S' 'Z' 'P']
['F' 'U']
['F' 'D' 'C' 'S' 'Z']
['T' 'G' 'L' 'Z']
['T' 'G' 'Z' 'L']
['X' 'H' 'Q' 'O' 'C' 'F' 'P' 'D' 'Z']
['X' 'Q' 'H' 'C' 'O' 'F' 'Z' 'P' 'D']
['X' 'O' 'P' 'Z']
['P' 'U' 'O']
['Q' 'F' 'M' 'G' 'O']
['L' 'O' 'V' 'I' 'X' 'S' 'P' 'Z' 'C']
['T' 'O' 'B' 'W' 'Z' 'D']
['V' 'H' 'N' 'E' 'Q' 'X' 'M']


In [ ]:
#creating independant and dependatn variable tensors
from torch import tensor
import torch
import numpy as np


t_dep = tensor(doh.target_F)
dohp = doh.drop(columns=['target_F','target_Q'])

def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

t_indep = df_to_tensor(dohp)

In [ ]:
#testsplit of training data
from fastai.data.transforms import RandomSplitter
trn_split,val_split=RandomSplitter(seed=42)(doh)

trn_indep,val_indep = t_indep[trn_split],t_indep[val_split]
trn_dep,val_dep = t_dep[trn_split],t_dep[val_split]
len(trn_indep),len(val_indep)

trn_dep = trn_dep[:,None]
val_dep = val_dep[:,None]

In [ ]:
val_dep

tensor([[0],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]], dtype=torch.uint8)

In [ ]:
import torch.nn.functional as F



def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

def train_model(epochs=30, lr=0.01):
    torch.manual_seed(442)
    coeffs = init_coeffs()
    for i in range(epochs): one_epoch(coeffs, lr=lr)
    return coeffs

def update_coeffs(coeffs, lr):
    layers,consts = coeffs
    for layer in layers+consts:
        layer.sub_(layer.grad * lr)
        layer.grad.zero_()

def init_coeffs():
    hiddens = [10, 10]  # <-- set this to the size of each hidden layer you want
    sizes = [n_coeff] + hiddens + [1]
    n = len(sizes)
    layers = [(torch.rand(sizes[i], sizes[i+1])-0.3)/sizes[i+1]*4 for i in range(n-1)]
    consts = [(torch.rand(1)[0]-0.5)*0.1 for i in range(n-1)]
    for l in layers+consts: l.requires_grad_()
    return layers,consts

def calc_preds(coeffs, indeps):
    layers,consts = coeffs
    n = len(layers)
    res = indeps
    for i,l in enumerate(layers):
        res = res@l + consts[i]
        if i!=n-1: res = F.relu(res)
    return torch.sigmoid(res)

In [ ]:
n_coeff = t_indep.shape[1]
coeffs = torch.rand(n_coeff)-0.5

coeffs = train_model(epochs=300,lr=0.234)

0.484; 0.484; 0.483; 0.483; 0.482; 0.480; 0.473; 0.432; 0.489; 0.384; 0.513; 0.511; 0.507; 0.484; 0.297; 0.302; 0.484; 0.484; 0.484; 0.484; 0.484; 0.484; 0.483; 0.483; 0.483; 0.482; 0.481; 0.478; 0.465; 0.346; 0.429; 0.195; 0.192; 0.169; 0.134; 0.110; 0.102; 0.095; 0.087; 0.080; 0.073; 0.067; 0.062; 0.057; 0.054; 0.050; 0.048; 0.045; 0.043; 0.042; 0.040; 0.039; 0.037; 0.036; 0.035; 0.034; 0.034; 0.033; 0.032; 0.031; 0.031; 0.030; 0.029; 0.029; 0.028; 0.028; 0.027; 0.027; 0.026; 0.026; 0.026; 0.025; 0.025; 0.024; 0.024; 0.024; 0.023; 0.023; 0.023; 0.022; 0.022; 0.022; 0.021; 0.021; 0.021; 0.021; 0.020; 0.020; 0.020; 0.019; 0.019; 0.019; 0.019; 0.019; 0.018; 0.018; 0.018; 0.018; 0.018; 0.017; 0.017; 0.017; 0.017; 0.017; 0.016; 0.016; 0.016; 0.016; 0.016; 0.016; 0.015; 0.015; 0.015; 0.015; 0.015; 0.014; 0.014; 0.014; 0.014; 0.014; 0.014; 0.013; 0.013; 0.013; 0.013; 0.013; 0.013; 0.012; 0.012; 0.012; 0.012; 0.012; 0.012; 0.011; 0.011; 0.011; 0.011; 0.011; 0.011; 0.010; 0.010; 0.010; 0.010;

In [ ]:
def acc(coeffs): return (val_dep.bool()==(calc_preds(coeffs, val_indep)>0.5)).float().mean()

acc(coeffs)


tensor(0.9994)